## IMPORTING THE DEPENDENCIES 

In [1]:
import gym 
import random

In [2]:
env = gym.make('SpaceInvaders-v0', render_mode='human')
height,width,channels = env.observation_space.shape
actions = env.action_space.n 

C:\Users\ViVian\AppData\Roaming\Python\Python39\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(


In [3]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
#episodes = 5
#for episode in range(1,episodes+1):
#    state = env.reset()
#    done = False
#    score = 0

#    while not done:
#        #env.render()
#        action =  random.choice([0,1,2,3,4,5])
#        n_state,reward,done,info = env.step(action)
#        score+=reward
#    print('EPISODES:{} SCORE:{}'.format(episode,score))

#env.close()

## CREATE A DEEP LEARNING MODEL WITH KERAS 

In [5]:
import numpy as np
import tensorflow as tf 

In [6]:
def build_model(height,width,channels,actions):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Convolution2D(32,(8,8),strides=(4,4),activation='relu',input_shape=(3,height,width,channels)))
    model.add(tf.keras.layers.Convolution2D(64,(4,4),strides=(2,2),activation='relu'))
    model.add(tf.keras.layers.Convolution2D(64,(3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.Dense(256,activation='relu'))
    model.add(tf.keras.layers.Dense(actions,activation='relu'))
    return model

In [7]:
model = build_model(height,width,channels,actions)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten (Flatten)           (None, 67584)             0         
                                                                 
 dense (Dense)               (None, 512)               34603520  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 6)                 1

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy,EpsGreedyQPolicy

In [10]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [11]:
dqn = build_agent(model, actions)
dqn.compile(tf.keras.optimizers.Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

: 

: 